<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

In [3]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)


,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,17.254242,24.335997,21.860235,20.521412,23.297489,21.403745,...,24.820764,0.000000,18.649907,20.390006,23.433725,22.375325,19.142636,21.262645,22.411346,19.349150
1,CeeDee Lamb,7,WR,2.6,21.900622,21.189847,23.290443,24.204766,19.377584,22.552715,...,20.418042,21.977984,20.331985,24.105824,25.106469,17.903953,21.682742,25.121331,21.055318,23.062231
2,Tyreek Hill,6,WR,3.2,18.691477,22.425863,22.860813,23.652446,20.511810,0.000000,...,23.521253,19.947274,23.248202,19.190125,20.549866,21.693775,22.527658,21.791031,18.847376,22.970965
3,Bijan Robinson,12,RB,5.0,24.471973,16.970603,22.644335,19.002473,23.547115,24.314475,...,25.701670,16.611831,21.615770,22.259711,0.000000,22.098847,21.079784,24.078805,20.919951,21.759239
4,Breece Hall,12,RB,5.4,26.040844,23.469652,21.554681,14.417697,23.756178,21.359522,...,19.220304,20.759553,22.961519,21.455322,0.000000,18.669776,23.351128,24.496324,22.021689,20.091712
5,Amon-Ra St. Brown,5,WR,6.2,21.871683,21.288743,20.739268,24.034143,0.000000,22.268310,...,22.496311,24.166353,22.701539,22.035221,22.917227,19.522595,19.902926,21.522004,19.647383,20.458223
6,Ja'Marr Chase,12,WR,6.6,21.170974,17.496997,22.453508,21.856032,20.079196,24.921814,...,16.153652,22.485811,21.442234,21.196751,0.000000,22.572541,25.779973,21.336933,22.967411,16.570433
7,Justin Jefferson,6,WR,7.0,21.849524,20.954241,20.745133,23.374955,20.132272,0.000000,...,23.925630,21.781990,22.862209,23.542436,18.931704,21.875308,24.794120,20.857532,19.760799,20.792840
8,Saquon Barkley,5,RB,9.2,20.170982,19.560002,21.476100,22.795666,0.000000,19.387806,...,19.719891,21.926147,18.321076,19.911842,21.090966,21.697285,19.062637,16.476271,21.055470,23.227333
9,A.J. Brown,5,WR,10.2,21.220928,24.540313,24.930576,18.091084,0.000000,25.549554,...,21.669608,20.500167,18.669834,21.636011,24.343501,20.559944,17.549786,21.435401,21.119421,21.130541


In [5]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [6]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

# 1. Spieler-Liste und Positionszuordnung
players = df_small['Player'].tolist()
pos = dict(zip(df_small['Player'], df_small['POS']))

# 3. Weitere Parameter
weeks = list(range(1, 18))
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}

# 4. Beispielhafte Listen für bereits gedraftete Spieler (hier leer)
dm_players = []
opp_players = []

# 5. Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players} #
x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion
model.objective = (
    1.0 * xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1, 16)) +
    150.0 * xsum(z[t] for t in range(16, 18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

model += xsum(y[i] for i in players) <= 18

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i, t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung starten
model.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [7]:
drafted_players = [i for i in players if y[i].x >= 0.99]
print("Dein Team:")
for player in drafted_players:
    print(player)

# Zielfunktionswert (Gesamtpunkte)
print("Zielfunktionswert:", model.objective_value)

Dein Team:
Sam LaPorta
D'Andre Swift
Zamir White
Rhamondre Stevenson
Najee Harris
Jayden Daniels
Marquise Brown
Jordan Addison
Jared Goff
Brian Thomas Jr.
Jameson Williams
Khalil Shakir
Zach Charbonnet
Cairo Santos
Chig Okonkwo
New York Giants
Rashod Bateman
Jacksonville Jaguars
Zielfunktionswert: 0.0


In [34]:
### Parameter ###
# The set of NFL players and defensive teams.
players = df_small['Player'].tolist()

# The set of positions
positions = {"QB", "RB", "WR", "TE", "K", "DST"}

# The set of weeks in the NFL regular and playoff seasons
weeks = list(range(1, 18))

# Players and their position
pos = dict(zip(df_small['Player'], df_small['POS']))

# Position Limit: The upper bound on the number of starting players for position
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}

# The overall pick number of the DM’s k-th draft pick
n_k = 0

# The set of players that the DM has drafted by her k-th pick
dm_players = []

# The set of players that the Opponents have drafted by her k-th pick
opp_players = []

# Anticipated ranking of unselected player i at the DM’s k-th draft pick.
df_sorted = df_small.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
Rk = dict(zip(df_sorted["Player"], df_sorted["Rank"]))

# An estimate of the number of fantasy points player i will score in week t of the NFL regular season.
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}

# The predicted amount of fantasy points the DM needs so to be reasonably confident to win in week t ∈ T against the DM’s matchup opponent.
beta = {t: 100.0 for t in weeks}

# The number of players the DM must draft at position j
gamma = {"QB": 2, "RB": 3, "WR": 3, "TE": 2, "K": 1, "DST": 1}

alpha = 1.0
lambda_0 = 1
lambda_1 = 100
lambda_2 = 150

# Teams
number_of_teams = 6
teams = [f"Team {i+1}" for i in range(number_of_teams)]

# Snake Draft order
num_rounds = 15

draft_order = []
for r in range(num_rounds):
    if r % 2 == 0:
        draft_order.extend(teams)
    else:
        draft_order.extend(teams[::-1])

# Initiate empty rosters and available players
rosters = {team: [] for team in teams}
available_players = set(players)

# Initiate empty draft log
draft_log = []
drafted_players = []

# print all parameters
print("Players:", players)
print("Positions:", positions)
print("Weeks:", weeks)
print("Pos:", pos)
print("Pos Limit:", pos_limit)
print("n_k:", n_k)
print("dm_players:", dm_players)
print("opp_players:", opp_players)
print("Rk:", Rk)
print("f_dict:", f_dict)
print("beta:", beta)
print("gamma:", gamma)
print("teams:", teams)
print("draft_order:", draft_order)
print("rosters:", rosters)
print("available_players:", available_players)
print("draft_log:", draft_log)


Players: ['Christian McCaffrey', 'CeeDee Lamb', 'Tyreek Hill', 'Bijan Robinson', 'Breece Hall', 'Amon-Ra St. Brown', "Ja'Marr Chase", 'Justin Jefferson', 'Saquon Barkley', 'A.J. Brown', 'Jonathan Taylor', 'Garrett Wilson', 'Jahmyr Gibbs', 'Puka Nacua', 'Kyren Williams', 'Marvin Harrison Jr.', 'Travis Etienne Jr.', 'Derrick Henry', 'Isiah Pacheco', 'Davante Adams', 'Josh Allen', 'Travis Kelce', 'Drake London', 'Chris Olave', "De'Von Achane", 'Josh Jacobs', 'Nico Collins', 'Patrick Mahomes II', 'James Cook', 'Mike Evans', 'Jalen Hurts', 'Rachaad White', 'Sam LaPorta', 'Deebo Samuel Sr.', 'Brandon Aiyuk', 'Michael Pittman Jr.', 'Cooper Kupp', 'Lamar Jackson', 'Joe Mixon', 'Alvin Kamara', 'DK Metcalf', 'Kenneth Walker III', 'Jaylen Waddle', 'Stefon Diggs', 'DJ Moore', 'DeVonta Smith', 'Mark Andrews', 'Trey McBride', 'Malik Nabers', 'C.J. Stroud', 'Dalton Kincaid', 'James Conner', 'George Kittle', 'Amari Cooper', 'Anthony Richardson Sr.', 'Aaron Jones Sr.', 'George Pickens', "D'Andre Swift"

In [35]:
from mip import Model, BINARY, maximize, xsum

### Optimization ###
for pick_num, team in enumerate(draft_order):
    remaining_df = df_small[~df_small["Player"].isin(drafted_players)].sort_values("AVG").reset_index(drop=True)
    remaining_df["Rank"] = remaining_df.index + 1
    Rk = dict(zip(remaining_df["Player"], remaining_df["Rank"]))

    model = Model(sense=maximize, solver_name="CBC")

    # Entscheidungsvariablen
    y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players} #
    x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
    z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

    # Bereits gedraftete Spieler verbieten
    for i in players:
        if i not in available_players:
            model += y[i] == 0

    # Bereits im Team befindliche Spieler erzwingen
    for i in rosters[team]:
        model += y[i] == 1

    # Zielfunktion
    model.objective = (
        lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
        lambda_1 * xsum(z[t] for t in range(1, 16)) +
        lambda_2 * xsum(z[t] for t in range(16, 18))
    )

    # Constraints
    # Minimum amount of players per position
    for j in positions:
        model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
        for t in weeks:
            model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

    # Maximum roster size
    model += xsum(y[i] for i in players) <= num_rounds

    # Lineup logic -> only drafted players can be in lineup
    for i in players:
        for t in weeks:
            model += x[i, t] <= y[i]

    # Win condition
    for t in weeks:
        model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]


      # --- Robuste Draft-Constraint (1b) ---
    # Für alle zukünftigen Picks dieses Teams: Begrenze die Zahl der Top-Spieler, die noch gedraftet werden dürfen
    # (Paper: bis zum k̅-ten Pick max. k̅-k Spieler aus den Top α·(n_k̅-n_k) Spielern)
    # Hier: n_k = aktueller Pick, n_k̅ = zukünftiger Pick
    team_picks = [i for i, t in enumerate(draft_order) if t == team]
    k = team_picks.index(pick_num)
    n_k = pick_num + 1
    for future_pick_idx in range(k + 1, len(team_picks)):
        n_kbar = team_picks[future_pick_idx] + 1
        top_cut = int(alpha * (n_kbar - n_k))
        if top_cut > 0:
            top_players = [i for i in players if i in available_players and Rk.get(i, 99999) <= top_cut]
            model += xsum(y[i] for i in top_players) <= (future_pick_idx - k)

    # Optimierung
    status = model.optimize()
    if status != 0:
        print(f"Keine zulässige Lösung für Team {team} bei Pick {pick_num + 1}")
        break

    # Pick player that gets drafted (highest ranked player)
    picked_player = None
    candidates = [i for i in players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team] and i in available_players]
    if candidates:
        # Wähle den mit bestem (niedrigstem) aktuellen Ranking
        picked_player = min(candidates, key=lambda i: Rk.get(i, 99999))
    else:
        print(f"Kein Spieler für Team {team} gefunden bei Pick {pick_num + 1}")
        break

    # Spieler zum Roster hinzufügen und aus verfügbaren Spielern entfernen
    rosters[team].append(picked_player)
    available_players.remove(picked_player)
    drafted_players.append(picked_player)


    # Loggen
    round_num = pick_num // len(teams) + 1
    draft_log.append({"Pick": pick_num + 1, "Team": team, "Player": picked_player, "Round": round_num})

# DataFrame aus Log erzeugen
df_draft = pd.DataFrame(draft_log)
df_draft

Keine zulässige Lösung für Team Team 1 bei Pick 1


""


In [36]:
model = Model(sense=maximize)
y = {i: model.add_var(var_type=BINARY) for i in players}

for j in positions:
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
model += xsum(y[i] for i in players) <= 15

status = model.optimize()
print("Status:", status)
print("Draft:", [i for i in players if y[i].x is not None and y[i].x > 0.5])


Status: OptimizationStatus.OPTIMAL
Draft: ['Travis Kelce', 'Jalen Hurts', 'Trey McBride', "D'Andre Swift", 'Kyler Murray', 'Najee Harris', 'Marquise Brown', 'Khalil Shakir', 'Zach Charbonnet', 'Jake Moody', 'Green Bay Packers', 'Rashod Bateman']
